<a href="https://colab.research.google.com/github/juli275/IA_UdeA_2023_2/blob/main/Fake_Job_Posting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carga de Librerias y Datos

In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
url = 'https://media.githubusercontent.com/media/juli275/IA_UdeA_2023_2/main/fake_job_postings.csv'

In [ ]:
Dataset = pd.read_csv(url)
Dataset01 = Dataset.copy(deep=True)

#Visualizacion Datos de Kaggle y Filtración

In [ ]:
Dataset01

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


**Exploración de datos**

Eliminaremos aquellas caracteristicas que tengan 60% de los datos en null

In [ ]:
Dataset01.isnull().mean()<0.4

job_id                  True
title                   True
location                True
department             False
salary_range           False
company_profile         True
description             True
requirements            True
benefits               False
telecommuting           True
has_company_logo        True
has_questions           True
employment_type         True
required_experience     True
required_education     False
industry                True
function                True
fraudulent              True
dtype: bool

In [ ]:
Dataset01= Dataset01[Dataset01.columns[Dataset01.isnull().mean()<0.4]]

Se rellenará el null por un string vacío

In [ ]:
Dataset01.fillna('',inplace=True)

<ipython-input-27-a1eae898ce08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01.fillna('',inplace=True)


Se eliminaran algunas columnas de nuestro dataset debido a que no son muy útiles para lo que deseamos realizar

In [ ]:
Dataset01.drop(columns=['telecommuting','has_company_logo','has_questions','job_id'], inplace=True)

<ipython-input-28-9aec9857ddcc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01.drop(columns=['telecommuting','has_company_logo','has_questions','job_id'], inplace=True)


In [ ]:
Dataset01

,title,location,company_profile,description,requirements,employment_type,required_experience,industry,function,fraudulent
0,Marketing Intern,"US, NY, New York","We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Other,Internship,,Marketing,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland","90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Full-time,Not Applicable,Marketing and Advertising,Customer Service,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,,,,0
3,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Full-time,Mid-Senior level,Computer Software,Sales,0
4,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full-time,Mid-Senior level,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,"CA, ON, Toronto",Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,Full-time,Mid-Senior level,Computer Software,Sales,0
17876,Payroll Accountant,"US, PA, Philadelphia",WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Full-time,Mid-Senior level,Internet,Accounting/Auditing,0
17877,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,Full-time,,,,0
17878,Graphic Designer,"NG, LA, Lagos",,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Contract,Not Applicable,Graphic Design,Design,0


Sintetizaremos todas las características juntas en una columna "text" para posteriormente eliminar todas las otras columnas innecesarias.

In [ ]:
Dataset01['text'] = Dataset01['title']+" "+Dataset01['location']+" "+Dataset01['company_profile']+" "+Dataset01['description']+" "+Dataset01['requirements']+" "+Dataset01['employment_type']+" "+Dataset01['required_experience']+" "+Dataset01['industry']+" "+Dataset01['function']

<ipython-input-30-2138c9e343cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['title']+" "+Dataset01['location']+" "+Dataset01['company_profile']+" "+Dataset01['description']+" "+Dataset01['requirements']+" "+Dataset01['employment_type']+" "+Dataset01['required_experience']+" "+Dataset01['industry']+" "+Dataset01['function']


In [ ]:
Dataset01.drop(columns=['title','location','company_profile','description','requirements','employment_type','required_experience','industry','function'],inplace=True)

<ipython-input-31-64f4a7b0398d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01.drop(columns=['title','location','company_profile','description','requirements','employment_type','required_experience','industry','function'],inplace=True)


In [ ]:
Dataset01

,fraudulent,text
0,0,"Marketing Intern US, NY, New York We're Food52..."
1,0,"Customer Service - Cloud Video Production NZ, ..."
2,0,"Commissioning Machinery Assistant (CMA) US, IA..."
3,0,"Account Executive - Washington DC US, DC, Wash..."
4,0,"Bill Review Manager US, FL, Fort Worth SpotSou..."
...,...,...
17875,0,"Account Director - Distribution CA, ON, Toron..."
17876,0,"Payroll Accountant US, PA, Philadelphia WebLin..."
17877,0,Project Cost Control Staff Engineer - Cost Con...
17878,0,"Graphic Designer NG, LA, Lagos Nemsia Studios..."


Se removeran los saltos de línea, salto de linea y retorno, y los tabs

In [ ]:
Dataset01['text'] = Dataset01['text'].str.replace('\n', ' ')
Dataset01['text'] = Dataset01['text'].str.replace('\r', ' ')
Dataset01['text'] = Dataset01['text'].str.replace('\t', ' ')

<ipython-input-33-8786ec1f3745>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['text'].str.replace('\n', ' ')
<ipython-input-33-8786ec1f3745>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['text'].str.replace('\r', ' ')
<ipython-input-33-8786ec1f3745>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

También, los números y los caracteres especiales

In [ ]:
Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[0-9]',' ',x))
Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))

<ipython-input-34-069a21f2c95f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[0-9]',' ',x))
<ipython-input-34-069a21f2c95f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))


Y colocar todo el texto en minúsculas

In [ ]:
Dataset01['text']= Dataset01['text'].apply(lambda s:s.lower() if type(s) == str else s)

<ipython-input-35-a84bfa9843d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text']= Dataset01['text'].apply(lambda s:s.lower() if type(s) == str else s)


Se comprueba si el tipo de la variable es string y se divide el string en una lista de palabras

In [ ]:
Dataset01['text']= Dataset01['text'].apply(lambda s:" ".join(s.split()) if type(s) == str else s)

<ipython-input-36-2f8bb51e35b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text']= Dataset01['text'].apply(lambda s:" ".join(s.split()) if type(s) == str else s)


In [ ]:
Dataset01

,fraudulent,text
0,0,marketing intern us ny new york we're food and...
1,0,customer service cloud video production nz auc...
2,0,commissioning machinery assistant cma us ia we...
3,0,account executive washington dc us dc washingt...
4,0,bill review manager us fl fort worth spotsourc...
...,...,...
17875,0,account director distribution ca on toronto ve...
17876,0,payroll accountant us pa philadelphia weblinc ...
17877,0,project cost control staff engineer cost contr...
17878,0,graphic designer ng la lagos nemsia studios is...


#**STOPWORDS (Palabras vacías)**

Palabras vacías es el nombre que reciben las palabras sin significado como artículos, pronombres, preposiciones, etc. que son filtradas antes o después del procesamiento de datos en lenguaje natural (texto)

 Con las stopwords de nltk se limpiarán los datos de estas palabras que no agregan valor al modelo

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
Dataset01['text'] = Dataset01['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))
Dataset01

<ipython-input-39-806703c2068b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset01['text'] = Dataset01['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))


,fraudulent,text
0,0,marketing intern us ny new york we're food we'...
1,0,customer service cloud video production nz auc...
2,0,commissioning machinery assistant cma us ia we...
3,0,account executive washington dc us dc washingt...
4,0,bill review manager us fl fort worth spotsourc...
...,...,...
17875,0,account director distribution ca toronto vend ...
17876,0,payroll accountant us pa philadelphia weblinc ...
17877,0,project cost control staff engineer cost contr...
17878,0,graphic designer ng la lagos nemsia studios lo...


Con **one_hot()** se generará una representacion numerica de nuestro texto, dandole 5000 numeros de clases para el vocubulario.

Luego con **pad_sequences()** vamos a ajustar la secuencia a 40, rellenando las que son menores de 40.

Con se representará numericamente las descripciones de ofertas trabajo manteniendo la misma longitud.



In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

one_hot_x = [one_hot(description,5000) for description in Dataset01['text']]
max_l = 40
embedded_description = pad_sequences(one_hot_x,max_l)

#Metricas
Definiremos entonces las metricas necesarias para empezar a evaluar nuestro algoritmo

In [ ]:
import keras

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR')
]

#Modelo
Se realizará un modelo de red neuronal secuencial para el procesamiento de la oferta de trabajo, utilizando capas de incrustación, capas de LSTM bidireccionales, capas de dropout y capas densas para modelar

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(5000,40,'uniform',input_length=max_l))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=METRICS)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
X = np.array(embedded_description)
Y = np.array(Dataset01['fraudulent'])

Implementaremos validación cruzada

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=3)

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=6, batch_size=30)


Epoch 1/6
398/398 [==============================] - 26s 66ms/step - loss: 0.0080 - tp: 591.0000 - fp: 8.0000 - tn: 11302.0000 - fn: 19.0000 - accuracy: 0.9977 - precision: 0.9866 - recall: 0.9689 - auc: 0.9990 - prc: 0.9955 - val_loss: 0.2051 - val_tp: 126.0000 - val_fp: 55.0000 - val_tn: 5649.0000 - val_fn: 130.0000 - val_accuracy: 0.9690 - val_precision: 0.6961 - val_recall: 0.4922 - val_auc: 0.7875 - val_prc: 0.5234
Epoch 2/6
398/398 [==============================] - 26s 65ms/step - loss: 0.0037 - tp: 604.0000 - fp: 4.0000 - tn: 11306.0000 - fn: 6.0000 - accuracy: 0.9992 - precision: 0.9934 - recall: 0.9902 - auc: 0.9983 - prc: 0.9970 - val_loss: 0.1937 - val_tp: 140.0000 - val_fp: 86.0000 - val_tn: 5618.0000 - val_fn: 116.0000 - val_accuracy: 0.9661 - val_precision: 0.6195 - val_recall: 0.5469 - val_auc: 0.8227 - val_prc: 0.5499
Epoch 3/6
398/398 [==============================] - 27s 67ms/step - loss: 0.0024 - tp: 606.0000 - fp: 2.0000 - tn: 11308.0000 - fn: 4.0000 - accuracy: 0

In [ ]:
def predict(m,fake_job_post):
  one_hot_input = one_hot(fake_job_post,5000)
  embedded = pad_sequences([one_hot_input],maxlen=max_l)

  pred = m.predict(embedded)
  #print(pred)

  if(pred > 0.5):
    return "FAKE"
  else:
    return "TRUE"

#Prueba
Para la prueba se hizo una filtracion en el dataset siguiendo la columna 'fraudulent' y se aplico la funcion predict para un opcion aleatoria de ellos. Así, para los verdaderos la funcion entregara 'TRUE' y para los falsos 'FAKE'.

In [ ]:
import random as rd
true = Dataset01[Dataset01['fraudulent']==0].index

i = rd.choice(true)
predict(model,Dataset01.loc[i,'text'])

1/1 [==============================] - 0s 26ms/step


'TRUE'

In [ ]:
fraud = Dataset01[Dataset01['fraudulent']==1].index

j = rd.choice(fraud)
predict(model,Dataset01.loc[j,'text'])


1/1 [==============================] - 0s 29ms/step


'FAKE'